# Описание проекта

Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений.

В датасете содержатся данные пользователей, впервые совершивших действия в приложении после 7 октября 2019 года.

Датасет *mobile_dataset.csv* содержит колонки:
- event.time  — время совершения
- event.name  — название события
- user.id  — идентификатор пользователя

Датасет *mobile_sources.csv* содержит колонки:
- userId  — идентификатор пользователя
- source  — источник, с которого пользователь установил приложение

Расшифровки событий:
- advert_open  — открытие карточки объявления
- photos_show  — просмотр фотографий в объявлении
- tips_show  — пользователь увидел рекомендованные объявления
- tips_click  — пользователь кликнул по рекомендованному объявлению
- contacts_show  и  show_contacts  — пользователь нажал на кнопку "посмотреть номер телефона" на карточке объявления
- contacts_call  — пользователь позвонил по номеру телефона на карточке объявления
- map  — пользователь открыл карту размещенных объявлений
- search_1  —  search_7  — разные события, связанные с пос поиском по сайту
- favorites_add  — добавление объявления в избранное

**Задачи проекта:**

- Выделите группы пользователей, которые различаются по метрикам:
    - retention rate,
    - время проведенное в приложении,
    - частота действий,
    - конверсия с целевое действие - просмотр контактов
- Проведите исследовательский анализ данных
- Сегментируйте пользователей на основе действий
- Проверьте статистические гипотезы


# Декомпозиция

**1. Предобработка данных**

1.1. Загрузка данных и знакомство с ними
    
1.2. Привеление названий стобцов к змеиному регистру
    
1.3. Приведение данных в столбцах к нужным типам
    
1.4. Проверка на пропуски
    
1.5. Поверка на дубликаты
    
    
**2. Исследовательский анализ данных**
    
2.1. Выделение сессий пользователей

2.2. Расчет метрик:
   - Retention Rate 
   - Рассчёт сколько в среднем времени пользователи проводят в приложении
   - Рассчёт как часто пользователи совершают действия в приложении
   - Конверсия в целевое действие — просмотр контактов     


**3. Сегментация клиентов**
  
3.1. Рассчёт Retention rate для каждого сегмента

3.2. Сравнение сегментированных групп по количеству событий (действия пользователей)

3.3. Для каждого сегмента считаем конверсию в целевое событие (просмотр контактов)

    
**4. Проверка статистических гипотез**

4.1. Проверка гипотезы: две группы клиентов, пришедших по ссылкам из yandex и google, демонстрируют разную конверсию в просмотры контактов
    
4.3. Проверка собственной гипотезы: пользователи, добавившие объявление в избранное (*favorites_add*) чаще доходят до целевого действия - "посмотреть номер телефона" (*contacts_show  и  show_contacts*) на карточке объявления
    
**5. Выводы и рекомендации**

## Предобработка данных

**Импортируем бибилотеки и взглянем на имеющиеся датасеты**

In [ ]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math as mth
from datetime import timedelta
from datetime import datetime, timedelta
from scipy import stats as st
from cmath import sqrt
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
mobile_sourсes = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_soures.csv')
mobile_dataset = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')

In [ ]:
display(mobile_sourсes.head())
display(mobile_dataset.head())

In [ ]:
mobile_sourсes.info()

In [ ]:
mobile_sourсes

In [ ]:
mobile_dataset.info()

**Приведём название столбцов к змеиному регистру**

In [ ]:
mobile_sourсes.rename(columns={'userId': 'user_id'}, inplace=True)

In [ ]:
mobile_dataset.columns = mobile_dataset.columns.str.replace('.','_')
mobile_sourсes.columns = mobile_sourсes.columns.str.replace('.','_')

**Изменим тип данных в *event_time***

In [ ]:
mobile_dataset['event_time'] = pd.to_datetime(mobile_dataset['event_time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
mobile_dataset

**Проведём проверку на пропуски**

In [ ]:
mobile_dataset.isna().sum()

In [ ]:
mobile_sourсes.isna().sum()

**Проверим датасеты на явные и неявные дубликаты**

In [ ]:
mobile_dataset.duplicated().sum()

In [ ]:
mobile_sourсes.duplicated().sum()

Явных дубликатов нет. Теперь посмотрим что там с неявными

In [ ]:
mobile_dataset['event_name'].unique()

Чтож, заменим *show_contacts* на *contacts_show*, т.к. это одно и тоже, а *search_1*, *search_2* и т.д. на просто *search*

In [ ]:
mobile_dataset['event_name'] = mobile_dataset['event_name'].replace('show_contacts', 'contacts_show')

In [ ]:
mobile_dataset['event_name'] = mobile_dataset['event_name'].replace(['search_1', 'search_2', 'search_3', 'search_4', 'search_5',
                                                           'search_6', 'search_7'], 'search')

In [ ]:
mobile_dataset['event_name'].unique()

In [ ]:
mobile_sourсes['source'].unique()

**Объединим два датасета в один по id пользователей**

In [ ]:
mobile_dataset['user_id'].nunique()

In [ ]:
mobile_sourсes['user_id'].nunique()

In [ ]:
df = mobile_dataset.merge(mobile_sourсes, on='user_id') 

In [ ]:
df

**Промежуточный вывод по предобработке данных:**

- Импортировали бибилотеки
- Загрузили данные и ознакомились с ними с ними
- Привели названия стобцов к правильному синтаксису
- Привели event_time к нужному типу данных
- Проверили наличие пропусков
- Проверили наличие явных дубликатов
- Выявили и обработали неявные дубликаты
- Объединили данные в один датасет для исследования

In [ ]:
df['date'] = df['event_time'].dt.date

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time']).dt.round('s')

## Исследовательский анализ данных

### Выделим сессии пользователей

In [ ]:
g = (df.sort_values(['user_id', 'event_time']).groupby('user_id')['event_time'].diff() > pd.Timedelta('30Min')).cumsum()

df['session'] = df.groupby(['user_id', g], sort=False).ngroup() + 1

In [ ]:
df

In [ ]:
df.sort_values(['user_id', 'event_time'])

### Расчитаем метрики

#### Retention rate

Для начала добавим столбец с датами и приведём его к нужному нам формату

In [ ]:
df['date'] = df['event_time'].dt.date
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.head()

А теперь посмотрим данные за какой период хранятся в нашем датасете

In [ ]:
display(df['date'].min())
display(df['date'].max())

Значится у нас имеются события с 7 октября 2019 по 3 ноября 2019. Примерно месяц, следовательно будет удобно посчитать 4 когорты по неделям. Что ж начнём с самого начала, а именно с первого дня активности пользователей, для чего сделаем отдельный датафрейм *activity_date_first*

In [ ]:
activity_date_first = df.groupby(['user_id'])['date'].min()
activity_date_first.name = 'activity_date_first'
user_activity = df.join(activity_date_first,on = 'user_id')

Теперь найдём первую неделю активности, она же будет нашей первой когортой

In [ ]:
user_activity['activity_week'] = pd.to_datetime(user_activity['date'],
                                                unit ='d') - pd.to_timedelta(user_activity['date'].dt.dayofweek, unit = 'd')
user_activity['first_activity_week'] = pd.to_datetime(user_activity['activity_date_first'],
                                                      unit = 'd') - pd.to_timedelta(
user_activity['activity_date_first'].dt.dayofweek, unit = 'd')

Рассчитаем *lifetime* пользователя когорт

In [ ]:
user_activity['cohort_lifetime'] = user_activity['activity_week'] - user_activity['first_activity_week']
user_activity['cohort_lifetime'] = user_activity['cohort_lifetime'] / np.timedelta64(1,'W')
user_activity['cohort_lifetime'] = user_activity['cohort_lifetime'].astype(int)

Посчитаем размеры когорт

In [ ]:
cohort_sizes = user_activity.groupby(['first_activity_week','cohort_lifetime'], as_index = False
                                    ).agg({'user_id':'nunique'})

Итак, у нас есть почти все данные для рассчёта Retention Rate, 
Однако нужно сперва получить число пользователей, которые изначально были в когорте, чтобы на него разделить число пользователей в каждую следующую неделю. 

In [ ]:
initial_users_count = cohort_sizes[cohort_sizes['cohort_lifetime'] == 0][['first_activity_week','user_id']]
initial_users_count = initial_users_count.rename(columns={'user_id':'original_cohort_users'})
cohort_sizes = cohort_sizes.merge(initial_users_count,on='first_activity_week')

Наконец мы можем рассчитать нашу метрику

In [ ]:
cohort_sizes['retention'] = cohort_sizes['user_id']/cohort_sizes['original_cohort_users']

Построим сводную таблицу удержания

In [ ]:
retention_pivot = cohort_sizes.pivot_table(index='first_activity_week',
                                      columns='cohort_lifetime',values='retention',aggfunc='sum').fillna(0)
display(retention_pivot)

Наглядней будет рассмотреть на тепловой карте

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(retention_pivot, annot = True, fmt = '.2%', cmap = "Blues")
plt.title('Тепловая карта удержания')  
plt.yticks(rotation = 45) 
plt.xlabel('Лайфтайм', fontsize = 14)
plt.ylabel('Дата начала когорты', fontsize = 14)
plt.show()

**Промежуточный вывод:**

- Показатель удержания падает после первой недели больше, чем на половину. Мало что можно сказать о клиентах, пришедших после 21 октября. К концу первой недели больше половины уникальных пользователей перестает пользоваться приложением.

- Чем больше когорта проводит времени в приложении, тем меньше пользователей остаётся, что печально, хоть и вполне закономерно. Не всякий находит приложение удобным и полезным для себя. Это может быть связано с тем, что в нашу цифровую эпоху большинство пользователей установив приложение, могут просто забыть про него уже через час, если у них не возникает необходимости постоянной необходимости пиробретать товары с не сильно высокой ликвидностью(т.е. не товары повседневного использования вроде продуктов питания).

#### Сколько в среднем времени пользователи проводят в приложении

Теперь нам нужно выделить время начала и конца пользовательских сессий и глянуть на среднюю продолжительность сессий

In [ ]:
df['session_day'] = df['event_time'].dt.day

In [ ]:
first_event_session = df.groupby(['user_id', 'session_day']).agg({'event_time': ['min', 'max']}).reset_index()
display(first_event_session)

In [ ]:
first_event_session.columns = [''.join(col) for col in first_event_session.columns]

In [ ]:
display(first_event_session)

In [ ]:
first_event_session['session_duration_sec'] = (
    first_event_session['event_timemax'] - first_event_session['event_timemin']).dt.seconds
display(first_event_session['session_duration_sec'].mean())

Среднее время сессий составило 4982 секунды. Это примерно 1,38 от часа. Теперь нужно взглянуть на среднее время по пользователям

In [ ]:
user_session_duration = first_event_session.groupby('user_id').agg({'session_duration_sec': 
                        'mean'}).reset_index().sort_values(by='session_duration_sec', ascending=False)

In [ ]:
user_session_duration['session_duration_sec'].describe()

In [ ]:
plt.figure(figsize = (12, 6))
user_session_duration['session_duration_sec'].hist(bins = 50, range = (0, 9000))
plt.title('Соотношение количества сессий к продолжительности')
plt.xlabel('Продолжительность сессии', fontsize = 15)
plt.ylabel('Количество сессий', fontsize = 15)
plt.show()

Пик в районе нуля может обуславливаться техническими ошибками как в приложении, так и в выгружнных данных. Информативней будет взглянуть на медианное значение составляющее около 16,43 от минуты, что в свою очередь сильно меньше среднего значения. Но похоже что этого времени хватит чтобы зайти в приложение и выбрать товар, который изначально искал

**Промежуточный вывод:**

- В среднем сессии длятся в районе 1 часа и 20 минут. Посмотрим на среднее время по пользователям
- Данные про 0 секунд в приложении можно отнести в техническим ошибкам, а слишком длительное времяприпровождение пользователя в приложении, либо к технической ошибке при выгрузке данных, либо предположить, что это суммарное время сессий за разные дни.
- Медиана сильно меньше среднего значения, и составляет около 17 минут, что ещё раз позволяет нам убедиться, что от 20 до 30 минут достаточно пользователю на одну сессию.

#### Частота действий

In [ ]:
event_named = df.sort_values(by = 'event_name', ascending = False)
event_named

In [ ]:
event_named.reset_index().groupby('event_name')['user_id'].nunique()

Сразу видно что наиболее частое действие в приложении это tips_show, т.е. показ рекомендованного объявления. Это может говорить о том, что в приложении можно грамотно разместить то, что необходимо(например рекламу или важное объявление). Теперь взглянем нп количество действий в рамках отдельной сессии

In [ ]:
event_session = df.groupby(['user_id', 'session_day']).agg({'event_time': ['min', 'max'], 'event_name':'count'}).reset_index()
event_session.head()

In [ ]:
event_session.columns = [''.join(col) for col in event_session.columns]
event_session = event_session.sort_values(by = 'event_namecount', ascending = False)

In [ ]:
event_session['event_namecount'].describe()

In [ ]:
plt.figure(figsize = (10, 6))
event_session['event_namecount'].hist(bins = 20, range=(0, 250))
plt.title('Соотношение действий на сессию')
plt.xlabel('Кол-во действий', fontsize = 15)
plt.ylabel('Частота действий', fontsize = 15)
plt.show()

**Промежуточный вывод:**

- В основном пользователи делают от 10 до 20 действий в приложении
- Наибольшее число выбросов в районе 0, связанные с технической частью вопроса. И до 384 действий в большую сторону.
- Среднее количество действий за сессию около 9.5, а медианное в районе 6.

#### Конверсия с целевое действие - просмотр контактов

In [ ]:
df_contacts = df.query('event_name == "contacts_show"')
df_contacts.head()

In [ ]:
(df_contacts['user_id'].nunique() / df['user_id'].nunique()) * 100

**Промежуточный вывод:**

Конверсия в целевое действие — 'contacts_show' (просмотр контактов) находится в районе 23%. Не хорошо, но и не ужасно

**Вывод по исследовательскому анализу:**

- Наш retention составляет около 10,5% за 28 дней. Поведение пользователей находится в рамках нормы.
- В среднем пользователь проводит в приложении от 15 до 25 минут в месяц. При поиске конкретного товара такой промежуток времени должен быть достаточным.
- Самое популярное действие 'tips_show', однако 'tips_click' на предпоследнем месте. Стоит обратить на это внимание, т.к. объявления, которые приложение рекомендует не соответствуют запросам потребителя.
- Конверсия по 'просмотру контакта' находится в районе 23%. Если обратить вниание на алгоритмы рекомендаций, то то и "показ контактов" тоже наверняка возрастёт

## Сегментация клиентов

Проведём кластеризацию по времени проведения в приложении

In [ ]:
df.sort_values(['user_id', 'event_time'])

In [ ]:
scaler = StandardScaler() 
df_cluster = pd.pivot_table(df, index = 'user_id'\
                            ,columns = ['event_name'],values = 'session_day',fill_value=0)\
.rename_axis(None, axis=1)

display(df_cluster.head())

Уберём действия, которые мало характеризуют клиента, т.к. являются навязанными рекламодателем(платформой)

In [ ]:
x = df_cluster.drop(['tips_show', 'tips_click'], axis = 1)
display(x.head())

In [ ]:
x = df_cluster.drop(['tips_show', 'tips_click'], axis = 1)
display(x.head())

In [ ]:
x_sc = scaler.fit_transform(x)  
linked = linkage(x_sc, method = 'ward')
plt.figure(figsize= (25, 10))  
dendrogram(linked, orientation = 'top', no_labels = True)
plt.title('Дендрограмма иерархической кластеризации')
plt.show() 

Визуально выделяем 2 кластера, стало быть минимальное количество кластеров для обучения модели кластеризации 2. Обучим модель кластеризации и глянем что получилось

In [ ]:
sc = StandardScaler()
x_sc_1 = sc.fit_transform(x)

km = KMeans(n_clusters = 2, random_state = 0)
labels =  km.fit_predict(x_sc)

In [ ]:
df_cluster['cluster'] = labels
display(df_cluster.groupby('cluster').mean())

In [ ]:
df_cluster.reset_index().groupby('cluster')['user_id'].nunique()

In [ ]:
df_cluster.groupby('cluster').mean().T

**Промежуточный вывод:**

- Кластеризация провели методом KMeans. Количество кластеров определили как 3

- Характеристика кластеров:
    - Кластер "0" - самый многочисленный (3365 пользователей). Больше используют карту размещения объявлений и так же пользуются поиском. Видимо здесь тоже преобладает конкретика при поиске товаров, но так же важен вопрос логистики(ведь никто не захочет плестись за нужной ему вещью к чёрту на куличики, если можно преобрести такой же в каком нибудь хоз. магазине неподалёку, пусть и дороже)
    - Кластер "1" - самый малочисленный (928 пользователей). Чаще всего рассматривают фотографии товара и пользуются функциями поиска. Можно предположить, что пользователи в этом сегменте препочитают искать конкретные товары и довольно большое значение придают внешщему представлению товара.

- Таким образом можем приступить к тем вычислениям, которые покажут нам, на каком сегменте стоит строить развитие приложения в будущем

    3.1. Рассчёт Retention rate для каждого сегмента

    3.2. Сравнение сегментированных групп по количеству событий (действия пользователей)

    3.3. Для каждого сегмента считаем конверсию в целевое событие (просмотр контактов)

### Рассчёт Retention rate для каждого сегмента

In [ ]:
cluster_id = pd.pivot_table(df_cluster, values = 'cluster',index='user_id').reset_index()
display(cluster_id.head())

In [ ]:
df = df.merge(cluster_id, how = 'left', on = 'user_id')

display(df_cluster)

In [ ]:
df = df_cluster.merge(user_activity, how = 'left', on = 'user_id')

In [ ]:
display(df_cluster)

Выведем количество пользователей в кластерах и создадим колонку с минимальной датой события

In [ ]:
df.groupby('cluster').agg({'user_id':'nunique'})

In [ ]:
df['min'] = df.groupby('user_id')['event_time'].transform('min')

Теперь создадим переменные для каждого кластера, чтобы рассчитать Retention rate по ним

In [ ]:
retention_rate_0 = df.query('cluster==0')
retention_rate_1 = df.query('cluster==1')

Можем вычислить размеры когорт, исходное количество пользователей, рассчитывать показатель Retention rate и построить хитмэп

Кластер "0"

In [ ]:
cohort_sizes = retention_rate_0.groupby(['first_activity_week','cohort_lifetime'], as_index = False
                                    ).agg({'user_id':'nunique'})

initial_users_count = cohort_sizes[cohort_sizes['cohort_lifetime'] == 0][['first_activity_week','user_id']]
initial_users_count = initial_users_count.rename(columns={'user_id':'original_cohort_users'})
cohort_sizes = cohort_sizes.merge(initial_users_count,on='first_activity_week')

cohort_sizes['retention'] = cohort_sizes['user_id']/cohort_sizes['original_cohort_users']
retention_pivot = cohort_sizes.pivot_table(index='first_activity_week',
                                      columns='cohort_lifetime',values='retention',aggfunc='sum').fillna(0)

plt.figure(figsize=(12, 8))
plt.title('Тепловая карта удержания пользователей 0 кластера')
sns.heatmap(retention_pivot, annot=True, fmt='.1%', linewidths=2)
plt.show()

Кластер "1"

In [ ]:
cohort_sizes = retention_rate_1.groupby(['first_activity_week','cohort_lifetime'], as_index = False
                                    ).agg({'user_id':'nunique'})

initial_users_count = cohort_sizes[cohort_sizes['cohort_lifetime'] == 0][['first_activity_week','user_id']]
initial_users_count = initial_users_count.rename(columns={'user_id':'original_cohort_users'})
cohort_sizes = cohort_sizes.merge(initial_users_count,on='first_activity_week')

cohort_sizes['retention'] = cohort_sizes['user_id']/cohort_sizes['original_cohort_users']
retention_pivot = cohort_sizes.pivot_table(index='first_activity_week',
                                      columns='cohort_lifetime',values='retention',aggfunc='sum').fillna(0)

plt.figure(figsize=(12, 8))
plt.title('Тепловая карта удержания пользователей 1 кластера')
sns.heatmap(retention_pivot, annot=True, fmt='.1%', linewidths=2)
plt.show()

**Промежуточный вывод:**

- Retention rate по кластерам:
    - 0 кластер: при 3365 пользователях, RR = 8,9%
    - 1 кластер: при 928 пользователях, RR = 21,6%

Таким образом соотношение количества пользователей и показателя RR лучше всего у 1 кластера. 

### Сравнение сегментированных групп по количеству событий (действия пользователей)

Снова выделим отдельные кластеры для расчёта метрики

In [ ]:
cluster_0 = df.query('cluster==0')
cluster_1 = df.query('cluster==1')

Посчитаем количество каждого действия и частоту этого действия по кластерам

In [ ]:
cluster_0_event_count = (cluster_0.groupby('event_name', as_index=False)
                      .agg(event_count=('event_name', 'count'))
                      .sort_values(by='event_count', ascending=False))

cluster_0_event_count['event_count'] = round(cluster_0_event_count['event_count']/cluster_0_event_count['event_count'].sum(), 3)

print()
display('Частота событий в кластере 0',cluster_0_event_count)

In [ ]:
cluster_1_event_count = (cluster_1.groupby('event_name', as_index=False)
                      .agg(event_count=('event_name', 'count'))
                      .sort_values(by='event_count', ascending=False))

cluster_1_event_count['event_count'] = round(cluster_1_event_count['event_count']/cluster_1_event_count['event_count'].sum(), 3)

print()
display('Частота событий в кластере 1',cluster_1_event_count)

**Промежуточный вывод:**

- Частота событий по кластерам:
    - 0 кластер: топ 3 event_count: tips_show, advert_open, search
    - 1 кластер: топ 3 event_count: photo_show, search, tips_show 


Очевидно, что большинство кластеров увидело рекомендованные объявления (tips_show), однако видно, что далеко не все перешли по этим рекомендациям. Зато многие открывали использовали поиск по сайту. Это может говорить о том, что большинство пользователей ищут что то конкретное, нежели заходят в приложение посмотреть чего там в принципе есть. Однако стоит всё же обратить внимание на рекомендации, уж больно хороший инструмент продажи и используется с такой низкой эффективностью

### Для каждого сегмента считаем конверсию в целевое событие (просмотр контактов)

Напишем функцию для рассчёта конверсии в целевое событие и используем уже разделённые на кластеры данные

In [ ]:
def get_cs (y):
    x = y.query('event_name == "contacts_show"')
    cr = (x['user_id'].nunique() / y['user_id'].nunique()) * 100
    print('{:.2f}%'.format(cr))

In [ ]:
print('Конверсия в целевое действие (просмотр контактов) по кластерам')

print('Пользователи из 0 кластера:')
get_cs(cluster_0)

print('Пользователи из 1 кластера:')
get_cs(cluster_1)

**Вывод по сегментации пользователей приложения:**

- Итак, на основании исследования охараткеризуем каждый кластер:

    - Кластер 0: состоит из 3365 пользователей, имеет коэфициент удержаний в 8,9%, часто открывают карточку товара. Имеет конверсию в целевое действие в 18.22%;
    - Кластер 1: состоит из 928 пользователей, имеет коэфициент удержаний в 21,6%, смотрит фотокарточки товара или ищет что то конкретное. Имеет конверсию в целевое действие в 39.66%;
   
Таким образом, можно преположить, что наилучшим образом для приложения может сложиться фокусировка на инересах 1 кластера, как кластера с хорошим процентов удержания и конверсией в целевое действие.

## Проверка статистических гипотез

### Проверка гипотезы: две группы клиентов, пришедших по ссылкам из yandex и google, демонстрируют разную конверсию в просмотры контактов

Н0: Конверсия в просмотры контактов у пользователей, привлеченных из yandex и google одинаковая 

Н1: Конверсия в просмотры контактов у пользователей, привлеченных из yandex и google разная

In [ ]:
yandex = df.query('source == "yandex"')
google = df.query('source == "google"')
sample_yandex = pd.pivot_table(yandex.query('event_name == "contacts_show" and source !="other"')\
                                    ,index = 'event_name', values= ['user_id'],aggfunc = {'user_id':'nunique'}).reset_index()
sample_google = pd.pivot_table(google.query('event_name == "contacts_show" and source !="other"')\
                                    ,index = 'event_name', values= ['user_id'],aggfunc = {'user_id':'nunique'}).reset_index()
action = np.array([sample_yandex.loc[0,'user_id']
                  ,sample_google.loc[0,'user_id']]) 
total = np.array([len(pd.Series(yandex['user_id'].unique())),len(pd.Series(google['user_id'].unique()))])

In [ ]:
def z_test_1(action, total):
    alpha = .05
    p1 = action[0]/total[0]
    p2 = action[1]/total[1]
    p_combined = (action[0] + action[1]) / (total[0] + total[1])

    difference = p1 - p2 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / total[0] + 1 / total[1]))
    
    distr = st.norm(0, 1) 
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / total[0] + 1 / total[1]))

    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print('p-значение: ', p_value)
    if p_value < alpha:
        print('Отвергаем нулевую гипотезу: между выборками есть значимая разница')
    else:
        print(
            'Не получилось отвергнуть нулевую гипотезу, нет оснований считать конверсию разной'
        ) 

In [ ]:
z_test_1(action, total)

**Промежуточный вывод:**

Гипотеза, что "две группы клиентов, пришедших по ссылкам из yandex и google, демонстрируют разную конверсию в просмотры контактов" не подтвердилась

### Проверка собственной гипотезы: пользователи, добавившие объявление в избранное (favorites_add) чаще доходят до целевого действия - "посмотреть номер телефона" (contacts_show и show_contacts) на карточке объявления

Потрачено

----

Н0: Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, не демонстрируют статистически важных различий

Н1:Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, демонстрируют разную конверсию в просмотры контактов

----

Н0: Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, не демонстрируют статистически важных различий

Н1: Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, демонстрирует статистически важные различия

In [ ]:
hypot_2 = df.query('event_name == "favorites_add"')['user_id'].unique()

id_group_A = df.query('user_id in @hypot_2')['user_id'].unique()
df_group_A = df.query('user_id in @id_group_A')
id_target_A = df_group_A.query('event_name == "contacts_show"')['user_id'].nunique()

id_group_B = df.query('user_id not in @hypot_2')['user_id'].unique()
df_group_B = df.query('user_id in @id_group_B')
id_target_B = df_group_B.query('event_name == "contacts_show"')['user_id'].nunique()

print("Количество уникальных пользователей в группе А:", len(id_group_A))
print("Количество событий в группе А:", len(df_group_A))
print("Количество уникальных пользователей группы А, совершивших целевое действие:", id_target_A)
print("Количество уникальных пользователей в группе B:", len(id_group_B))
print("Количество событий в группе B:", len(df_group_B))
print("Количество уникальных пользователей группы В, совершивших целевое действие:", id_target_B)

In [ ]:
def z_test_3(grpA_2, grpB_2, grpA_1, grpB_1, alpha=0.05):
    p1 = grpA_2 / grpA_1
    p2 = grpB_2 / grpB_1
    
    print(grpA_2, grpB_2, grpA_1, grpB_1)

    p_combined = (grpA_2 + grpB_2) / (grpA_1 + grpB_1)
    difference = p1 - p2
    
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / grpA_1 + 1 / grpB_1))
    distr = st.norm(0, 1)  
    p_value = (1 - distr.cdf(abs(z_value))) * 2 
    print('p-значение: ', p_value)
    if (p_value < alpha): 
        print('Отвергаем нулевую гипотезу, между выборками есть статистически значимые различия')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет') 

In [ ]:
z_test_3(id_target_A, id_target_B, len(id_group_A), len(id_group_B))

**Промежуточный вывод:**


Гипотеза, что "Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, не демонстрируют статистически важных различий" не подтвердилась

## Общие выводы и рекомендации

**Мы выполнили изначальные задачи проекта, а именно:**

- Провели предобработку имеющих данных
- Провели исследовательский анализ данных
- Выделили группы пользователей, которые различаются по метрикам:
    - retention rate,
    - время проведенное в приложении,
    - частота действий,
    - конверсия с целевое действие - просмотр контактов
- Сегментировали пользователей на основе действий
- Выделили сегменты пользователей, которые различаются по метрикам:
    - retention rate
    - по количеству событий (действия пользователей)
    - конверсии в целевое событие (просмотр контактов)
- Проверили статистические гипотезы:
    - Две группы клиентов, пришедших по ссылкам из yandex и google, демонстрируют разную конверсию в просмотры контактов
    - Конверсия в просмотры контактов между теми пользователями, которые добавляют объявления в закладки, от тех кто не добавляет, не демонстрируют статистически важных различий

На основе проведённых исследований, можно сказать, что коэфициент удержания пользователей можно считать приемлимым, но для дальнейшего развития следует улучшить этот показатель. Для этого можно провести более масштабную рекламную компанию, или улучшить само приложение(например, доработать алгоритм рекомендованных объявлений, чтобы больше пользователей переходили по ним)

Так же при выборе кластера, на который стоит ориентироваться, следует обратить внимание на кластер № 1 из исследования, т.к. у него хороший процент удержания и конверсия в целевое действие.

Более того, т.к. гипотеза, что "две группы клиентов, пришедших по ссылкам из yandex и google, демонстрируют разную конверсию в просмотры контактов" не подтвердилась, то не стоит делать ставку на один из источников привлечения без дополнительных исследований. Но стоит обратить внимание на пользователей, которые добавляют товар "в избранное", т.к. их конверсия в просмотр контактов и последующую покупку может быть выше.